In [1]:
import pandas as pd
import json
import emoji

# Function to decode emojis in text
def decode_emojis(text):
    if isinstance(text, str):
        return emoji.emojize(text, language='alias')
    return text

# Load the CSV data into a DataFrame
input_file_csv = 'instagram_messages.csv'  # Replace with your CSV file path
df = pd.read_csv(input_file_csv)

# Initialize a list to hold the JSON data
json_data = []
df

,message_id,folder_name,sender_name,timestamp,message,share,reactions
0,1,akshatsumanp_797842791844050,Ritisha Katiyar,1689430610550,Heyy\n\nCollege n all kidhar ka plan hai ?,no,NaN
1,2,akshatsumanp_797842791844050,Akshat Suman P.,1689624121050,hii\n\numm...no decisions yet\n\ni don't think...,no,NaN
2,3,akshatsumanp_797842791844050,Ritisha Katiyar,1689627907963,Oh damn\n\nWhy tho ?\n\nCET n all kisi ke thro...,no,NaN
3,4,akshatsumanp_797842791844050,Akshat Suman P.,1689627992203,noo results not cool\n\nbtw u takin through cet?,no,NaN
4,5,akshatsumanp_797842791844050,Ritisha Katiyar,1689628177295,Oh i see\n\nNhi actually I wanted only COEP n ...,no,NaN
...,...,...,...,...,...,...,...
19060,3858,rishi_736861041275559,Ritisha Katiyar,1728209671877,Abeyy\n\nToh thik ho jaldi\n\nAur neeche aa,no,NaN
19061,3859,rishi_736861041275559,Rishi,1728210075489,ye\n\nLesgoo\n\nhehe,no,NaN
19062,3860,rishi_736861041275559,Ritisha Katiyar,1728213308770,replied to story/sent a reel: You sent an atta...,yes,:growing_heart:
19063,3861,rishi_736861041275559,Rishi,1728213495296,"""is John wick real:eyes::eyes:""\n\nतुमच्या मॅस...",no,NaN


In [2]:
# Find the most frequent sender_name
most_frequent_sender = df['sender_name'].value_counts().idxmax()
frequency = df['sender_name'].value_counts().max()
# Find the most frequent sender_name
most_frequent_sender = df['sender_name'].value_counts().idxmax()
most_frequent_sender_first_name = most_frequent_sender.split()[0]  # Extract the first name

print(f"The most frequent sender is: {most_frequent_sender} with {frequency} messages")


The most frequent sender is: Ritisha Katiyar with 9155 messages


In [3]:

# Iterate through the DataFrame to create the prompt-response pairs
for i in range(len(df) - 1):
    current_row = df.iloc[i]
    next_row = df.iloc[i + 1]
    
    if current_row['sender_name'] == most_frequent_sender or next_row['sender_name'] != most_frequent_sender:
        continue
    
    prompt_text = current_row['message']
    response_text = next_row['message']
    
    # Check if either the prompt or response is NaN
    if pd.isna(prompt_text) or pd.isna(response_text):
        continue
    
    json_data.append({
        "user": decode_emojis(prompt_text),
        most_frequent_sender_first_name: decode_emojis(response_text)
    })


In [4]:

# Convert the list to JSON format and save directly to output file
output_file = 'final_dataset.json'  # Replace with your desired output file path

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(json_data, f, ensure_ascii=False, indent=4)

print(f"Data has been saved to {output_file}")


Data has been saved to final_dataset.json


In [5]:
import json
import os

def json_to_jsonl(input_file, output_file):
    # Read the JSON data
    with open(input_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Write JSON objects to a JSON Lines file
    with open(output_file, 'w', encoding='utf-8') as f:
        for item in data:
            json.dump(item, f, ensure_ascii=False)
            f.write('\n')

# Example usage:
input_file = 'final_dataset.json'  # Replace with your input JSON file path
output_file = 'final_dataset.jsonl'  # Replace with your desired output JSON Lines file path

json_to_jsonl(input_file, output_file)


In [6]:
import json

input_file = 'final_dataset.jsonl'  # Replace with your actual JSONL file name
output_file = 'final_dataset2.jsonl'     # Replace with desired output file name

# Open input and output files
with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8') as outfile:
    for line in infile:
        # Parse JSON object from each line
        data = json.loads(line.strip())
        
        # Modify keys
        data['Prompt'] = data.pop('user', None)
        data['Completion'] = data.pop(most_frequent_sender_first_name, None)
        
        # Write modified JSON object to output file
        json.dump(data, outfile, ensure_ascii=False)
        outfile.write('\n')
